# 🏟️ Football Match Prediction Model

---
## GitHub Repository & Dataset
- **Project Source Code:** [ml_football_prediction](https://github.com/baladhamitul/ml_football_prediction)  
- **Dataset (matches.csv):** [Download/View here](https://github.com/baladhamitul/ml_football_prediction/blob/main/matches.csv)


## 📌 Project Overview

An **end-to-end machine learning pipeline** designed to predict football match outcomes as a binary classification task:
- **Class 1:** Win/Draw (WD) — Team avoids losing
- **Class 2:** Loss (L) — Team loses the match

---

## 🎯 Key Features

### Feature Engineering
- **Rolling Form Statistics** (5-match rolling windows):
  - Points, goals for/against, expected goals (xG)
  - Shot and shot-on-target counts
  - Win/loss rates over recent matches

- **Opponent Context**:
  - Opponent's rolling form features
  - Team vs. opponent difference features (relative strength)

- **Temporal Features**:
  - Year, month, day, and day-of-week extracted from match date

### Data Preprocessing Pipeline
- **Numeric columns**: Median imputation → Standard scaling
- **Categorical columns**: Most-frequent imputation → One-hot encoding
- **Leakage prevention**: Removes post-match statistics and actual outcomes

---

## 🤖 Model Training & Evaluation

**Algorithms Compared:**
1. Logistic Regression
2. SGD Classifier
3. Linear SVC
4. Random Forest
5. Extra Trees
6. Histogram Gradient Boosting

**Hyperparameter Tuning:**
- GridSearchCV with 5-fold stratified cross-validation
- Multiple metrics tracked: Accuracy, Balanced Accuracy, F1-Score

**Best Model:**
- Refit on full dataset
- Saved as `.joblib` file for production deployment

---

## 📊 Metrics Tracked

| Metric | Purpose |
|--------|---------|
| **Accuracy** | Overall correctness |
| **Balanced Accuracy** | Handles class imbalance |
| **F1-Score (WD)** | Precision-recall balance for "not losing" prediction |



## Step 1: Library Imports

The initial step involves importing Python packages needed for data manipulation, visualization, and machine learning.



In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier

from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    classification_report,
    f1_score,
    make_scorer,
)

import joblib



### Step 2: Load and Prepare Data

**Goal:** Read the match dataset, clean it, and define a supervised-learning target that the model can learn reliably.

At first, I framed the task as a **3-class classification** problem (`W`, `D`, `L`). However, the model achieved only around **~40% accuracy**, mainly because predicting three outcomes is harder and the boundaries between *win vs draw* (and sometimes *draw vs loss*) are often subtle and noisy in real match data. To reduce this ambiguity and improve learnability, I redesigned the label into a **binary target**: **`WD` (Win or Draw)** vs **`L` (Loss)**. This change made the objective more practical (“will the team avoid losing?”) and increased performance not only in **accuracy**, but also in metrics like **F1-score**, because the model could focus on separating *losses* from *non-losses*.

**Main actions:**
- Parse the `date` column and drop rows with invalid dates.
- Create a binary target: `WD` (Win/Draw) vs `L` (Loss).
- Add `pts` (W=3, D=1, L=0) for rolling form features.
- Sort by team/season/date so rolling windows are chronological.




In [ ]:
CSV_PATH = "matches.csv"
df = pd.read_csv(CSV_PATH)

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.dropna(subset=["date"]).copy()

df["target"] = df["result"].astype(str).replace({"W": "WD", "D": "WD", "L": "L"})

pts_map = {"W": 3, "D": 1, "L": 0}
df["pts"] = df["result"].map(pts_map)

df = df.sort_values(["team", "season", "date"]).reset_index(drop=True)

df.head()


,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,sh,sot,dist,fk,pk,pkatt,season,team,target,pts
0,1,2020-09-12,12:30,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,...,13.0,5.0,13.6,2.0,0.0,0.0,2021,Arsenal,WD,3
1,2,2020-09-19,20:00,Premier League,Matchweek 2,Sat,Home,W,2.0,1.0,...,6.0,3.0,15.3,0.0,0.0,0.0,2021,Arsenal,WD,3
2,4,2020-09-28,20:00,Premier League,Matchweek 3,Mon,Away,L,1.0,3.0,...,4.0,3.0,15.3,0.0,0.0,0.0,2021,Arsenal,L,0
3,6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,...,6.0,5.0,16.7,0.0,0.0,0.0,2021,Arsenal,WD,3
4,7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,...,11.0,3.0,18.2,2.0,0.0,0.0,2021,Arsenal,L,0


## Step 3: Create Rolling Form Features

**Goal:** define *how* we compute “recent form”.

We will compute rolling averages over the last **N previous matches** (not including the current match). This is important to avoid **data leakage**.

Examples:
- average points in last N matches
- average goals for/against in last N matches
- win-rate / loss-rate in last N matches


In [42]:
ROLL_N = 5

num_for_form = ["pts", "gf", "ga", "xg", "xga", "sh", "sot"]
num_for_form = [c for c in num_for_form if c in df.columns]

def add_rolling(group: pd.DataFrame) -> pd.DataFrame:
    """Add rolling 'form' features for one (team, season) group.

    We shift by 1 so the current match does NOT use its own outcome/statistics.
    """
    g = group.copy()

    for c in num_for_form:
        g[f"{c}_avg_last{ROLL_N}"] = g[c].shift(1).rolling(ROLL_N, min_periods=1).mean()

    g[f"winrate_last{ROLL_N}"] = (g["result"].shift(1) == "W").rolling(ROLL_N, min_periods=1).mean()
    g[f"lossrate_last{ROLL_N}"] = (g["result"].shift(1) == "L").rolling(ROLL_N, min_periods=1).mean()

    return g


## Step 4: Rolling Features Calculation

**Goal:** actually compute the rolling form features for each team.

**How:**
- Group by `team` and `season`.
- Apply the `add_rolling` function.
- Collect all rolling columns into `roll_cols`.
- Fill remaining missing values (early matches in a season) with dataset means.


In [43]:
df = df.groupby(["team", "season"], group_keys=False).apply(add_rolling)

roll_cols = [
    c for c in df.columns
    if (f"_avg_last{ROLL_N}" in c) or (f"_last{ROLL_N}" in c)
]

if roll_cols:
    df[roll_cols] = df[roll_cols].fillna(df[roll_cols].mean(numeric_only=True))

df[["team", "season", "date", "result", "target"] + roll_cols].head()


C:\Users\SIS\AppData\Local\Temp\ipykernel_19304\53253362.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["team", "season"], group_keys=False).apply(add_rolling)


,team,season,date,result,target,pts_avg_last5,gf_avg_last5,ga_avg_last5,xg_avg_last5,xga_avg_last5,sh_avg_last5,sot_avg_last5,winrate_last5,lossrate_last5
0,Arsenal,2021,2020-09-12,W,WD,1.363901,1.346185,1.394321,1.300185,1.336002,12.101741,4.027556,0.000000,0.00
1,Arsenal,2021,2020-09-19,W,WD,3.000000,3.000000,0.000000,1.800000,0.200000,13.000000,5.000000,0.500000,0.00
2,Arsenal,2021,2020-09-28,L,L,3.000000,2.500000,0.500000,1.600000,1.050000,9.500000,4.000000,0.666667,0.00
3,Arsenal,2021,2020-10-04,W,WD,2.000000,2.000000,1.333333,1.466667,1.766667,7.666667,3.666667,0.500000,0.25
4,Arsenal,2021,2020-10-17,L,L,2.250000,2.000000,1.250000,1.200000,1.375000,7.250000,4.000000,0.600000,0.20


## Step 5: Merge Opponent Rolling Features

**Goal:** for each match row, attach the opponent’s rolling form features.

**Idea:**
- Create a copy of the dataset with `team` and `opponent` swapped.
- Rename rolling columns as `opp_<feature>`.
- Merge back on `season/date/comp/team/opponent` so each match gets both sides’ form.


In [44]:
opp_cols = ["season", "date", "comp", "team", "opponent"] + roll_cols
df_opp = df[opp_cols].copy()

df_opp = df_opp.rename(columns={"team": "opponent", "opponent": "team"})
df_opp = df_opp.rename(columns={c: f"opp_{c}" for c in roll_cols})

df2 = df.merge(
    df_opp,
    on=["season", "date", "comp", "team", "opponent"],
    how="left"
)

for c in roll_cols:
    oc = f"opp_{c}"
    if oc in df2.columns:
        df2[oc] = df2[oc].fillna(df2[c])

df2.head()


,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,lossrate_last5,opp_pts_avg_last5,opp_gf_avg_last5,opp_ga_avg_last5,opp_xg_avg_last5,opp_xga_avg_last5,opp_sh_avg_last5,opp_sot_avg_last5,opp_winrate_last5,opp_lossrate_last5
0,1,2020-09-12,12:30,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,...,0.00,1.363901,1.346185,1.394321,1.300185,1.336002,12.101741,4.027556,0.000000,0.00
1,2,2020-09-19,20:00,Premier League,Matchweek 2,Sat,Home,W,2.0,1.0,...,0.00,3.000000,3.000000,0.000000,1.800000,0.200000,13.000000,5.000000,0.500000,0.00
2,4,2020-09-28,20:00,Premier League,Matchweek 3,Mon,Away,L,1.0,3.0,...,0.00,3.000000,3.000000,1.500000,2.950000,0.800000,18.500000,4.500000,0.666667,0.00
3,6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,...,0.25,2.000000,2.000000,1.333333,1.466667,1.766667,7.666667,3.666667,0.500000,0.25
4,7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,...,0.20,1.333333,2.000000,2.333333,1.433333,1.766667,17.333333,4.666667,0.250000,0.25


## Step 6: Create Difference Features

**Goal:** capture *relative* strength/form between the team and its opponent.

For each rolling feature `c`, we create:
- `diff_c = team_c − opp_c`

This often helps models because it directly encodes “who is stronger right now?”.


In [45]:
for c in roll_cols:
    oc = f"opp_{c}"
    if oc in df2.columns:
        df2[f"diff_{c}"] = df2[c] - df2[oc]

diff_cols = [c for c in df2.columns if c.startswith("diff_")]
df2[diff_cols].head()


,diff_pts_avg_last5,diff_gf_avg_last5,diff_ga_avg_last5,diff_xg_avg_last5,diff_xga_avg_last5,diff_sh_avg_last5,diff_sot_avg_last5,diff_winrate_last5,diff_lossrate_last5
0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
2,0.000000,-0.5,-1.000000,-1.350000,0.250000,-9.000000,-0.500000,0.00,0.00
3,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
4,0.916667,0.0,-1.083333,-0.233333,-0.391667,-10.083333,-0.666667,0.35,-0.05


## Step 7: Build Feature Matrix and Target

**Goal:** create:
- `y`: the label we want to predict (`WD` vs `L`)
- `X`: the feature table

**Leakage prevention:** remove columns that directly reveal the outcome (like `result`) or are post-match stats.

**Extra features:** extract year/month/day/day-of-week from the match date, then drop the raw datetime column.


In [46]:
TARGET_COL = "target"
y = df2[TARGET_COL].astype(str)

drop_cols = [
    "result", "target", "pts",
    "gf", "ga", "xg", "xga", "sh", "sot",
    "match report", "notes"
]
X = df2.drop(columns=[c for c in drop_cols if c in df2.columns], errors="ignore")

X["date_year"] = df2["date"].dt.year
X["date_month"] = df2["date"].dt.month
X["date_day"] = df2["date"].dt.day
X["date_dow"] = df2["date"].dt.dayofweek
X = X.drop(columns=["date"], errors="ignore")

print("X shape:", X.shape)
print("y distribution:")
print(y.value_counts(dropna=False))


X shape: (1389, 49)
y distribution:
target
WD    841
L     548
Name: count, dtype: int64


## Step 8: Train-Test Split and Preprocessing Pipeline

**Goal:** make a robust preprocessing pipeline that works for mixed data types.

**Preprocessing rules:**
- Numeric columns: median imputation + standard scaling
- Categorical columns: most-frequent imputation + one-hot encoding

This is wrapped in a scikit-learn `Pipeline`, so any model we plug in sees a consistent feature representation.


In [47]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

prep = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ]), selector(dtype_include=np.number)),
        ("cat", Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]), selector(dtype_exclude=np.number)),
    ],
    remainder="drop"
)


## Step 9: Initial GridSearchCV with Accuracy Scoring

**Goal:** define a first (simple) hyperparameter search setup.

We:
- Build a pipeline: `prep -> model`.
- Define a parameter grid that can swap different models.
- Define stratified cross-validation so class balance is preserved in each fold.

(We will fit the full GridSearch in Step 12.)


In [49]:
pipe = Pipeline([
    ("prep", prep),
    ("model", LogisticRegression(max_iter=5000)),
])

param_grid = [
    {
        "model": [LogisticRegression(max_iter=5000, solver="lbfgs")],
        "model__C": [0.1, 0.3, 1.0, 3.0],
    },
    {
        "model": [LogisticRegression(max_iter=5000, solver="lbfgs")],
        "model__C": [0.1, 0.3, 1.0, 3.0],
        "model__class_weight": [None, "balanced"],
    },

    {
        "model": [SGDClassifier(loss="log_loss", random_state=42)],
        "model__alpha": [1e-4, 1e-3, 1e-2],
        "model__penalty": ["l2", "l1"],
        "model__class_weight": [None, "balanced"],
    },

    {
        "model": [LinearSVC(random_state=42)],
        "model__C": [0.3, 1.0, 3.0],
        "model__class_weight": [None, "balanced"],
    },

    {
        "model": [RandomForestClassifier(random_state=42)],
        "model__n_estimators": [200, 500],
        "model__max_depth": [None, 10, 20],
        "model__min_samples_leaf": [1, 3, 5],
        "model__class_weight": [None, "balanced"],
    },

    # Extra Trees
    {
        "model": [ExtraTreesClassifier(random_state=42)],
        "model__n_estimators": [300, 600],
        "model__max_depth": [None, 10, 20],
        "model__min_samples_leaf": [1, 3, 5],
        "model__class_weight": [None, "balanced"],
    },

    {
        "model": [HistGradientBoostingClassifier(random_state=42)],
        "model__max_depth": [None, 6, 10],
        "model__learning_rate": [0.05, 0.1],
        "model__max_iter": [200, 500],
    },
]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

gs_acc = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring="accuracy",
    refit=True,
    cv=cv,
    n_jobs=-1,
    verbose=1,
)

## Step 10: Enhanced GridSearchCV with Multiple Metrics

**Goal:** evaluate models with more than one metric.

We will track:
- **accuracy** (refit metric)
- **balanced accuracy** (helps if classes are imbalanced)
- **F1 for the `WD` class** (focuses on correctly identifying “not losing”)

We keep `refit='acc'` so the stored `best_estimator_` is the one with the best accuracy.


In [50]:
scoring = {
    "acc": "accuracy",
    "bal_acc": "balanced_accuracy",
    "f1_wd": make_scorer(f1_score, pos_label="WD"),
}

gs = GridSearchCV(
    pipe,
    param_grid=param_grid,
    scoring=scoring,
    refit="acc",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    return_train_score=True,
)


## Step 11: Model Comparison Function

**Goal:** after running GridSearchCV, quickly compare the *best* configuration for each model family.

The function below:
- Converts `cv_results_` into a DataFrame
- Extracts the model name from the `param_model`
- Selects useful metrics
- Returns the best row per model (sorted by test accuracy)


In [52]:
def gridsearch_model_comparison(gs: GridSearchCV) -> pd.DataFrame:
    res = pd.DataFrame(gs.cv_results_)

    res["model_name"] = res["param_model"].apply(lambda m: m.__class__.__name__)

    keep_cols = [
        "model_name",
        "mean_test_acc", "std_test_acc",
        "mean_test_bal_acc", "std_test_bal_acc",
        "mean_test_f1_wd", "std_test_f1_wd",
        "mean_fit_time",
        "params",
    ]
    keep_cols = [c for c in keep_cols if c in res.columns]
    res = res[keep_cols].copy()

    best_per_model = (
        res.sort_values("mean_test_acc", ascending=False)
           .groupby("model_name", as_index=False)
           .head(1)
           .sort_values("mean_test_acc", ascending=False)
           .reset_index(drop=True)
    )

    return best_per_model

## Step 12: Fit GridSearchCV and Generate Model Comparison

**Goal:** run cross-validated hyperparameter search and summarize performance.

This can take a while depending on:
- your dataset size
- how large the grid is
- how many CPU cores you have


In [53]:
gs.fit(X_train, y_train)
comparison = gridsearch_model_comparison(gs)

comparison


Fitting 5 folds for each of 114 candidates, totalling 570 fits


c:\Users\SIS\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
60 fits failed out of a total of 570.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\SIS\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\SIS\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\SIS\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 420, in fit
    self._final_estimator.fit(Xt, y,

,model_name,mean_test_acc,std_test_acc,mean_test_bal_acc,std_test_bal_acc,mean_test_f1_wd,std_test_f1_wd,mean_fit_time,params
0,LinearSVC,0.656187,0.037859,0.628619,0.036349,0.727376,0.034190,0.424462,"{'model': LinearSVC(C=0.3, random_state=42), '..."
1,SGDClassifier,0.655282,0.013973,0.632665,0.010739,0.721693,0.019270,0.153791,"{'model': SGDClassifier(loss='log_loss', rando..."
2,LogisticRegression,0.650782,0.026116,0.619789,0.024407,0.726256,0.025057,0.642873,"{'model': LogisticRegression(max_iter=5000), '..."
3,ExtraTreesClassifier,0.649877,0.023020,0.570050,0.026967,0.766472,0.013214,5.181345,{'model': ExtraTreesClassifier(random_state=42...
4,RandomForestClassifier,0.647166,0.019622,0.572589,0.024571,0.760772,0.010660,2.476766,{'model': RandomForestClassifier(random_state=...
5,HistGradientBoostingClassifier,NaN,NaN,NaN,NaN,NaN,NaN,0.109305,{'model': HistGradientBoostingClassifier(rando...


## Step 13: Extract Best Model Details

**Goal:**
- Print the best cross-validation score and parameters
- Evaluate on the held-out test split
- Retrain on the full dataset
- Save the trained pipeline as a `.joblib` file

Because we use a pipeline, the saved artifact includes both preprocessing and the chosen model.


In [54]:
print("Best CV accuracy:", gs.best_score_)
print("Best params:", gs.best_params_)

best_model = gs.best_estimator_

pred = best_model.predict(X_test)

print("\nTEST accuracy:", accuracy_score(y_test, pred))
print("TEST balanced acc:", balanced_accuracy_score(y_test, pred))
print("TEST F1 (WD):", f1_score(y_test, pred, pos_label="WD"))

labels = ["WD", "L"]
print("\nConfusion matrix (rows=true, cols=pred):\n", confusion_matrix(y_test, pred, labels=labels))
print("\nClassification report:\n", classification_report(y_test, pred, labels=labels, zero_division=0))

best_model.fit(X, y)
joblib.dump(best_model, "best_match_binary_model.joblib")
print("\nSaved -> best_match_binary_model.joblib")


Best CV accuracy: 0.6561871288328687
Best params: {'model': LinearSVC(C=0.3, random_state=42), 'model__C': 0.3, 'model__class_weight': None}

TEST accuracy: 0.697841726618705
TEST balanced acc: 0.6793831168831169
TEST F1 (WD): 0.7543859649122806

Confusion matrix (rows=true, cols=pred):
 [[129  39]
 [ 45  65]]

Classification report:
               precision    recall  f1-score   support

          WD       0.74      0.77      0.75       168
           L       0.62      0.59      0.61       110

    accuracy                           0.70       278
   macro avg       0.68      0.68      0.68       278
weighted avg       0.70      0.70      0.70       278



c:\Users\SIS\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(



Saved -> best_match_binary_model.joblib


c:\Users\SIS\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
